### Code section

##### Initialize SC server and MIDI input

In [1]:
import sc3nb as scn
import numpy as np
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process... Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... Done.


In [2]:
%sc MIDIClient.init;
%sc MIDIIn.connectAll;

MIDI: device 0 3 1832551344  (Microsoft GS Wavetable Synth)
MIDI: device 1 4 1832551352  (V61)
MIDI: device 2 5 1832551360  (MIDIOUT2 (V61))
MIDI Sources:
	MIDIEndPoint("V61", "V61")
	MIDIEndPoint("MIDIIN2 (V61)", "MIDIIN2 (V61)")
MIDI Destinations:
	MIDIEndPoint("Microsoft GS Wavetable Synth", "Microsoft GS Wavetable Synth")
	MIDIEndPoint("V61", "V61")
	MIDIEndPoint("MIDIOUT2 (V61)", "MIDIOUT2 (V61)")
-> MIDIClient
Requested notification messages from server 'sc3nb_remote'
sc3nb_remote: server process has maxLogins 8 - adjusting my options accordingly.
sc3nb_remote: keeping clientID (0) as confirmed by server process.
-> MIDIIn


##### Setup SC synths and global parameters

In [3]:
%%sc 
// Setting up general parameters used by the training tasks

~metronome_rates = [1, 0.75, 0.75, 0.75];
~metronome_amp = 0.05;
~metronome_amps = Pseq([~metronome_amp], inf);
~number_all_notes = 0;
~dur_per_beat = 1.0;


// Setting up SynthDefs used by the training tasks

~drumstick_buf = Buffer.read(s, "./sounds/drumstick.wav";);

// Metronome sound
SynthDef(\play_buf, {
    |out=0, buf, amp=0.5, rate=1|
    var sig = PlayBuf.ar(1, buf, BufRateScale.kr(buf) * rate, doneAction: 2) * amp;
    Out.ar(out, sig!2)
}).add;

// Generic piano sound the player hears when hitting keys on the keyboard
SynthDef(\train_note, { 
    |freq=440, amp=0.2, pan=0.5, gate=1, distortion_length=0, 
    noise_level=0, freq_distort_level=0, amp_reduction_level=0|
    var sig, env, note, noise, env_distortion;
    
    env = EnvGen.kr(Env.adsr(releaseTime: 0.5), gate, doneAction: Done.freeSelf);
    env_distortion = EnvGen.kr(Env.linen(0, distortion_length * ~dur_per_beat, 0, 1));
    
    note = LFTri.ar((1 - (noise_level * env_distortion)) * freq + (SinOsc.kr(20, 0, 15, 0) * freq_distort_level * env_distortion));
    
    sig = (noise_level * env_distortion) * BrownNoise.ar(0.25) + note;
    sig = sig * (amp - (amp_reduction_level * amp * env_distortion));
    sig = sig * env;
    Out.ar(0, 1-pan * sig);
    Out.ar(1, pan * sig);
}).add;

// Signal sound for Method 1
SynthDef(\default_note, { |freq=440, amp=0.2, sustain=0.5, pan=0.5|
    var sig;
    sig = LFTri.ar(freq, mul: amp) * EnvGen.kr(Env.perc(0.001, sustain), doneAction: Done.freeSelf);
    Out.ar(0, 1-pan * sig);
    Out.ar(1, pan * sig);
}).add;

// Signal sound for Method 2
SynthDef(\blip, { |freq=440, amp=0.2, sustain=0.5, pan=0.5|
    var sig;
    sig = Blip.ar(freq, mul: amp) * EnvGen.kr(Env.perc(0.001, sustain), doneAction: Done.freeSelf);
    Out.ar(0, 1-pan * sig);
    Out.ar(1, pan * sig);
}).add;

// Signal sound for Method 3
SynthDef(\tick, { |rate=2, amp=0.3, sustain=0.5, pan=0.5|
    var sig;
    sig = PlayBuf.ar(1, ~drumstick_buf, BufRateScale.kr(~drumstick_buf) * rate, doneAction: Done.freeSelf) * amp;
    Out.ar(0, 1-pan * sig);
    Out.ar(1, pan * sig);
}).add;

-> a SynthDef


In [4]:
%%sc
~notes = Array.newClear(128);    // array has one slot per possible MIDI note

MIDIdef.noteOn(\note_on, { 
    |veloc, num, chan, src, default_synth=\train_note|
    ~notes[num] = Synth(default_synth, [\freq, num.midicps, \amp, veloc * 0.00315]);
    }
);

MIDIdef.noteOff(\note_off, {
    |veloc, num, chan, src|
    ~notes[num].release;
    }
);

-> MIDIdef(note_off, noteOff, nil, nil, nil)


##### Setup SC functions

In [5]:
%%sc
// A function that plays different signal sounds depending on which sonification method
// is passed as argument.
~play_control_note = { |method, midi, pan|
                          switch(method,
                                1, { Synth(\default_note, [\freq, midi.midicps, \pan, pan]) },
                                2, { Synth(\blip, [\freq, midi.midicps, \pan, pan]) },
                                3, { Synth(\tick, [\freq, midi.midicps, \pan, pan]) })
                     };

// A function that returns a Task object which plays a set of notes in some rhythm.
// This function and the control task returned by it are used to realize interactive
// sonification feedback for notes played too late.
// The Task plays along the training rhythm in silent and produces a signal sound when
// the player has not played a note after the correct time for that note has passed. 
~setup_control_task = { |train_notes, train_rhythms, method, pan = 0.5|
                        Task({
                            var current_note, task_rhythm = Pseq(train_rhythms, inf).asStream;
                            current_note = 0;
                            loop {
                                train_notes.do({ |midi|
                                    if ((~train_inputs[current_note].size == 0), {
                                        ~play_control_note.value(method, midi, pan);
                                    });
                                    task_rhythm.next.wait;
                                    current_note = current_note + 1;
                                });
                            }
                        })
                      };

// Creates control task for left hand for two-hands tasks
~setup_control_task_left = { |train_notes, train_rhythms, method, pan = 0.5|
                        Task({
                            var current_note, task_rhythm = Pseq(train_rhythms, inf).asStream;
                            current_note = 0;
                            loop {
                                train_notes.do({ |midi|
                                    if ((~train_inputs_left[current_note].size == 0), {
                                        ~play_control_note.value(method, midi, pan);
                                    });
                                    task_rhythm.next.wait;
                                    current_note = current_note + 1;
                                });
                            }
                        })
                      };

// Creates control task for right hand for two-hands tasks
~setup_control_task_right = { |train_notes, train_rhythms, method, pan = 0.5|
                        Task({
                            var current_note, task_rhythm = Pseq(train_rhythms, inf).asStream;
                            current_note = 0;
                            loop {
                                train_notes.do({ |midi|
                                    if ((~train_inputs_right[current_note].size == 0), {
                                        ~play_control_note.value(method, midi, pan);
                                    });
                                    task_rhythm.next.wait;
                                    current_note = current_note + 1;
                                });
                            }
                        })
                      };

// Defines advanced MIDIdef for noteon messages (signals indicating that a key on the MIDI keyboard is pressed).
// Upon pressing a key, the time errors are calculated.
// The sonification feedback for early notes is activated when the player hits the key too early.
~setup_train_note_on = { |method|
                         MIDIdef(\note_on).free;
                         MIDIdef(\note_on_left).free;
                         MIDIdef(\note_on_right).free; 
                         MIDIdef.noteOn(\note_on, {
                             |veloc, num, chan, src|
                             var error_beat, error_beats, current_train_note;
                             ~notes[num] = Synth(\train_note, [\freq, num.midicps, \amp, veloc * 0.00315]);
                             error_beats =  ~train_clock.beats - ~solutions_time;
                             current_train_note = error_beats.minIndex({|item, i| item.abs});
                             error_beat = error_beats[current_train_note];
                             ~notes[num].set(\distortion_length, error_beat * (-1));
                             switch(method, 
                                    0, {},
                                    1, { ~notes[num].set(\amp_reduction_level, 0.8) },
                                    2, { ~notes[num].set(\freq_distort_level, 1) },
                                    3, { ~notes[num].set(\noise_level, 1) });
                             ~train_inputs[current_train_note] = ~train_inputs[current_train_note].add([num, ~train_clock.beats]);
                         });
                       };



// Defines advanced MIDIdef for noteon messages for two-hands tasks
~setup_train_note_on_2h = { |method|
                         MIDIdef(\note_on).free;
                         MIDIdef(\note_on_left).free;
                         MIDIdef(\note_on_right).free;    
                         MIDIdef.noteOn(\note_on_left, { // Define noteOn function for left note
                             |veloc, num, chan, src|
                             if( num == ~train_note_left, {
                                 var error_beat, error_beats, current_train_note;
                                 ~notes[num] = Synth(\train_note, [\freq, num.midicps, \amp, veloc * 0.00315, \pan, 0]);
                                 error_beats =  ~train_clock.beats - ~solutions_time_left;
                                 current_train_note = error_beats.minIndex({|item, i| item.abs});
                                 error_beat = error_beats[current_train_note];
                                 ~notes[num].set(\distortion_length, error_beat * (-1));
                                 switch(method, 
                                        0, {},
                                        1, { ~notes[num].set(\amp_reduction_level, 0.8) },
                                        2, { ~notes[num].set(\freq_distort_level, 1) },
                                        3, { ~notes[num].set(\noise_level, 1) });
                                 ~train_inputs_left[current_train_note] = ~train_inputs_left[current_train_note].add([num, ~train_clock.beats]);
                             });
                         });
                         MIDIdef.noteOn(\note_on_right, { // Define noteOn function for right note
                             |veloc, num, chan, src|
                             if( num == ~train_note_right, {
                                 var error_beat, error_beats, current_train_note;
                                 ~notes[num] = Synth(\train_note, [\freq, num.midicps, \amp, veloc * 0.00315, \pan, 1]);
                                 error_beats =  ~train_clock.beats - ~solutions_time_right;
                                 current_train_note = error_beats.minIndex({|item, i| item.abs});
                                 error_beat = error_beats[current_train_note];
                                 ~notes[num].set(\distortion_length, error_beat * (-1));
                                 switch(method, 
                                        0, {},
                                        1, { ~notes[num].set(\amp_reduction_level, 0.8) },
                                        2, { ~notes[num].set(\freq_distort_level, 1) },
                                        3, { ~notes[num].set(\noise_level, 1) });
                                 ~train_inputs_right[current_train_note] = ~train_inputs_right[current_train_note].add([num, ~train_clock.beats]);
                             });
                         });    
                       };

// Resets all MIDIdefs
~setup_default_note_on = {
                            MIDIdef(\note_on).free;
                            MIDIdef(\note_on_left).free;
                            MIDIdef(\note_on_right).free;
                            MIDIdef.noteOn(\note_on, { 
                                |veloc, num, chan, src|
                                ~notes[num] = Synth(\train_note, [\freq, num.midicps, \amp, veloc * 0.00315]);
                                }
                            );
                         }

-> a Function


##### Python classes and functions

In [6]:
class TrainingTask:
    """
    A class used to represent a rhythm training task.
    
    Attributes
    ----------
    signature: str
        the time signature for this task
    hands: int
        indicates whether the task is played with one hand or both hands
    notes: list of int
        a list of the notes to be played in the task given as MIDI notation
    rhythms: list of float
        a list of the time intervals between the notes
    offbeats: list of float
        a list containing the offsets (for both hands if hands == 2) 
        between start of the first note and beginning of the first bar
    beats_per_bar: int
        number of beats contained in one bar
    dur_per_beat: float
        duration of one beat measured in seconds
    """
    
    def __init__(self, signature, hands, notes, rhythms, offbeats):
        """
        Parameters
        ----------
        signature: str
            the time signature for this task
        hands: int
            indicates whether the task is played with one hand or both hands
        notes: list of int
            a list of the notes to be played in the task given as MIDI notation
        rhythms: list of float
            a list of the time intervals between the notes
        offbeats: list of float
            a list containing the offsets (for both hands if hands == 2) 
            between start of the first note and beginning of the first bar
        """
        self.signature = signature
        self.hands = hands
        self.notes = notes
        self.rhythms = rhythms
        self.offbeats = offbeats
        self.beats_per_bar = int(signature.split('/')[0])
        self.dur_per_beat = 4 / int(signature.split('/')[1])

class TrainingData:
    """
    A class used to store training data for one playthrough.
    
    Attributes
    ----------
    task_name: str
        name of the TrainingTask that is played in this playthrough
    task: TrainingTask
        the TrainingTask object named by task_name
    method: int
        index of sonification method used in this playthrough
    bpm: int
        tempo (in beats-per-minute) to be followed in this playthrough
    run_number: int
        index of the playthrough
    solutions_note: list of int
        list of correct note numbers for every note played in the playthrough
    solutions_time: list of float
        list of correct time values for every note played in the playthrough
    train_inputs: list of list
        list of input data recorded from the player, an element in train_inputs
        should be in the form [note number in midi (int), note time value (float),
        index of slot which the note belongs to (int)]
    absolute_errors: list of float
        list of absolute time errors, the deviation between note time values
        in solutions_time and note time values in train_inputs
    relative_errors: list of float, optional
        list of relative time errors, the deviation between time intervals
        between notes from solutions_time and from time data of train_input
    mean_abs: float
        mean value of the relative time errors
    std_abs: float
        standard deviation of the absolute time errors
    mear_rel: float
        mean value of the relative time errors
    std_rel: float
        standard deviation of the relative time errors
    """
    
    def __init__(self, task_name, task, method, bpm, run_number, solutions_note, solutions_time, train_inputs):
        """
        Parameters
        ----------
        task_name: str
            name of the TrainingTask that is played in this playthrough
        task: TrainingTask
            the TrainingTask object named by task_name
        method: int
            index of sonification method used in this playthrough
        bpm: int
            tempo (in beats-per-minute) to be followed in this playthrough
        run_number: int
            index of the playthrough
        solutions_note: list of int
            list of correct note numbers for every note played in the playthrough
        solutions_time: list of float
            list of correct time values for every note played in the playthrough
        train_inputs: list of list
            list of input data recorded from the player, an element in train_inputs
            should be in the form [note number in midi (int), note time value (float)]
        """
        
        self.task_name = task_name
        self.task = task
        self.method = method
        self.bpm = bpm
        self.run_number = run_number
        self.solutions_note = np.array(solutions_note)
        self.solutions_time = np.array(solutions_time)
        self.train_inputs = []
        for i in range(len(train_inputs)):
            for note in train_inputs[i]:
                note.append(i)
                self.train_inputs.append(note)
        self.train_inputs = np.array(self.train_inputs)
        self.absolute_errors = []
        self.relative_errors = []
        self.mean_abs = 0
        self.std_abs = 0
        self.mean_rel = 0
        self.std_rel = 0

In [7]:
import os
import pickle
import time

def setup_task_sc(task, settings, method):
    """Sets up global parameters for a playthrough.
    
    Parameters
    ----------
    task: TrainingTask
        the TrainingTask object which will be played in this playthrough,
        its attributes are inherited to control tempo of the metronome,
        one hand or two hands mode etc.
    settings: dict
        a dictionary containing setting parameters from the GUI
    method: int
        index of sonification method
    """
    
    global bpm, wait_bars, train_bars, allow_error, task_mode, beats_per_bar, dur_per_beat
    bpm = settings['bpm']
    wait_bars = settings['wait_bars']
    train_bars = settings['train_bars']
    allow_error = settings['allow_error']
    task_mode = settings['task_mode']
    beats_per_bar = task.beats_per_bar
    dur_per_beat = task.dur_per_beat
    %scs ~dur_per_beat = ^dur_per_beat
    setup_metronome_sc(task_mode)
    if task.hands == 1:
        setup_task_1h_sc(task, method)
    elif task.hands == 2:
        setup_task_2h_sc(task, method)
        
def setup_metronome_sc(mode):
    """Sets up metronome that will play along in a playthrough.
    
    Creates a SuperCollider Task object which plays a drumstick sound
    with a certain tempo and time signature.
    The volume of the metronome is controlled by the global sc variable
    ~metronome_amp.
    
    Parameters
    ----------
    mode: str
        if the mode is equal 'test', a special pattern of metronome sounds
        will be played so that with each passing bar the beats that can be
        heard are halved, else a continuously playing metronome is set
    """
    
    if mode == 'test':
        if beats_per_bar == 4:
            sc.lang.cmds(r"""
                ~metronome_amps = Pseq([~metronome_amp], ^wait_bars*^beats_per_bar) ++
                                  Pseq([~metronome_amp], ^beats_per_bar) ++ Pseq([~metronome_amp, 0], 2) ++
                                  Pseq([~metronome_amp], 1) ++ Pseq([0], ^beats_per_bar-1) ++
                                  Pseq([0], inf);
            """)
        else:
            sc.lang.cmds(r"""
                ~metronome_amps = Pseq([~metronome_amp], ^wait_bars*^beats_per_bar) ++
                                  Pseq([~metronome_amp], ^beats_per_bar) ++ 
                                  Pseq([~metronome_amp], 1) ++ Pseq([0], ^beats_per_bar-1) ++
                                  Pseq([0], inf);
            """) 
    else:
        %scs ~metronome_amps = Pseq([~metronome_amp], inf);
    sc.lang.cmds(r"""
        ~metronome = Task({
            var rate_pattern = Pseq(~metronome_rates.reshape(^beats_per_bar), ^wait_bars + ^train_bars).asStream;
            var amp_pattern = ~metronome_amps.asStream;
            loop {
                Synth(\play_buf, [buf: ~drumstick_buf, amp: amp_pattern.next, rate: rate_pattern.next]);
                1.wait;
            }
        });
    """)
        
def setup_task_1h_sc(task, method):
    """Sets up global parameters in SuperCollider environment 
    for a playthrough with one-hand task.
    
    Computes list of correct note numbers and list of correct
    note timings (starting at 0) for the entire playthrough.
    Sets up sonifications for the given method. ~setup_train_note_on
    is used to set up sonification feedback for early notes,
    ~control_task is used to set up sonification feedback for late notes.
    
    Parameters
    ----------
    task: TrainingTask
        the TrainingTask object which will be played
    method: int
        index of sonification method
    """
    
    global train_notes, train_rhythms, train_offbeat, task_mode, use_method
    train_notes = task.notes
    train_rhythms = task.rhythms
    train_offbeat = task.offbeats
    use_method = method
    sc.lang.cmds(r"""
        var playthrough_duration = ^train_offbeat, number_all_notes = 0, all_note_durs = Pseq(^train_rhythms, inf).asStream, timing_all_notes = List[];
        while({playthrough_duration < (^train_bars * ^beats_per_bar)}, {
            number_all_notes = number_all_notes + 1;
            timing_all_notes.add(playthrough_duration);
            playthrough_duration = playthrough_duration + all_note_durs.next;
            });
        ~number_all_notes = number_all_notes;
        ~timing_all_notes = timing_all_notes.array;
    """) 
    if task_mode == 'demo':
        sc.lang.cmds(r"""
            ~control_task = Task({
                var task_rhythm = Pseq(^train_rhythms, inf).asStream;
                loop {
                    ^train_notes.do({ |midi|
                    Synth(\default_note, [\freq, midi.midicps]);
                    task_rhythm.next.wait;
                    });
                }
            })
        """)
    elif task_mode == 'train':
        if method == 0:
            %scs ~control_task = Task({});
        else:
            %scs ~control_task = ~setup_control_task.value(^train_notes, ^train_rhythms, ^use_method);
        %scs ~setup_train_note_on.value(^use_method)
    elif task_mode == 'test':
        %scs ~setup_train_note_on.value(0)

def setup_task_2h_sc(task, method):
    """Sets up global parameters in SuperCollider environment 
    for a playthrough with two-hands task.
    
    Computes lists of correct note numbers and lists of correct
    note timings (starting at 0) for the entire playthrough for
    both left and right hand.
    Sets up sonifications for the given method for both hands. 
    ~setup_train_note_on_left and ~setup_train_note_on_right
    are used to set up sonification feedback for early notes,
    ~control_task_left and ~control_task_right are used to 
    set up sonification feedback for late notes.
    
    Parameters
    ----------
    task: TrainingTask
        the TrainingTask object which will be played
    method: int
        index of sonification method
    """
    
    global train_notes_left, train_notes_right, train_rhythms_left, train_rhythms_right, train_offbeat_left, train_offbeat_right, task_mode, use_method
    train_notes_left = task.notes[0]
    train_notes_right = task.notes[1]
    train_rhythms_left = task.rhythms[0]
    train_rhythms_right = task.rhythms[1]
    train_offbeat_left = task.offbeats[0]
    train_offbeat_right = task.offbeats[1]
    use_method = method
    %scs ~train_note_left = ^train_notes_left[0]
    %scs ~train_note_right = ^train_notes_right[0]
    sc.lang.cmds(r"""
        var playthrough_duration = ^train_offbeat_left, 
        number_all_notes_left = 0, all_note_durs_left = Pseq(^train_rhythms_left, inf).asStream, timing_all_notes_left = List[],
        number_all_notes_right = 0, all_note_durs_right = Pseq(^train_rhythms_right, inf).asStream, timing_all_notes_right = List[];
        while({playthrough_duration < (^train_bars * ^beats_per_bar)}, {
            number_all_notes_left = number_all_notes_left + 1;
            timing_all_notes_left.add(playthrough_duration);
            playthrough_duration = playthrough_duration + all_note_durs_left.next;
            });
        playthrough_duration = ^train_offbeat_right;
        while({playthrough_duration < (^train_bars * ^beats_per_bar)}, {
            number_all_notes_right = number_all_notes_right + 1;
            timing_all_notes_right.add(playthrough_duration);
            playthrough_duration = playthrough_duration + all_note_durs_right.next;
            });
        ~number_all_notes_left = number_all_notes_left;
        ~timing_all_notes_left = timing_all_notes_left.array;
        ~number_all_notes_right = number_all_notes_right;
        ~timing_all_notes_right = timing_all_notes_right.array;
    """)
    if task_mode == 'demo':
        sc.lang.cmds(r"""
            ~control_task_left = Task({
                var task_rhythm = Pseq(^train_rhythms_left, inf).asStream;
                loop {
                    ^train_notes_left.do({ |midi|
                    Synth(\default_note, [\freq, midi.midicps, \pan, 0]);
                    task_rhythm.next.wait;
                    });
                }
            });
            ~control_task_right = Task({
                var task_rhythm = Pseq(^train_rhythms_right, inf).asStream;
                loop {
                    ^train_notes_right.do({ |midi|
                    Synth(\default_note, [\freq, midi.midicps, \pan, 1]);
                    task_rhythm.next.wait;
                    });
                }
            });
        """)
    elif task_mode == 'train':
        if method == 0:
            %scs ~control_task_left = Task({});
            %scs ~control_task_right = Task({}); 
        else:
            %scs ~control_task_left = ~setup_control_task_left.value(^train_notes_left, ^train_rhythms_left, ^use_method, pan: 0);
            %scs ~control_task_right = ~setup_control_task_right.value(^train_notes_right, ^train_rhythms_right, ^use_method, pan: 1);
        %scs ~setup_train_note_on_2h.value(^use_method)
    elif task_mode == 'test':
        %scs ~setup_train_note_on_2h.value(0)

def play_task_sc(task):
    """Starts a playthrough.
    
    Creates a SuperCollider TempoClock, sets up the tempo
    using global parameters that are configured before and
    schedules the start of a one-hand task playthrough or
    two-hands task playthrough according to the respective
    attribute from the TrainingTask object.
    
    Parameters
    ----------
    task: TrainingTask
        the TrainingTask object which will be played in this
        playthrough
    """
    
    %scs ~train_clock = TempoClock.new
    %scs ~train_clock.tempo = ^bpm/60
    if task.hands == 1:
        play_task_1h_sc()
    elif task.hands == 2:
        play_task_2h_sc()            
            
def play_task_1h_sc():
    """Starts a one-hand task playthrough.
    
    Uses the TempoClock defined in previous function to schedule
    events.
    Starts the metronome and activates interactive sonification feedback.
    """
    
    sc.lang.cmds(r"""
        ~train_clock.schedAbs(~train_clock.beats.ceil, {
            ~train_inputs = ([]!~number_all_notes);
            ~solutions_time = ~timing_all_notes + ((~train_clock.beats + (^wait_bars * ^beats_per_bar)) ! ~number_all_notes);
            ~solutions_note = Pseq(^train_notes, inf).asStream.nextN(~number_all_notes);
            ~metronome.play(~train_clock);
            if((^task_mode != "test"), {
                ~train_clock.schedAbs(~train_clock.beats + (^wait_bars * ^beats_per_bar) + ^train_offbeat + ^allow_error, {
                    ~control_task.play(~train_clock);
                })
            });
            ~train_clock.sched((^wait_bars + ^train_bars) * ^beats_per_bar - 0.01, {
                ~metronome.stop;
            });      
            ~train_clock.sched((^wait_bars + ^train_bars) * ^beats_per_bar + ^allow_error - 0.01, {
                ~control_task.stop;  
                ~setup_default_note_on.value;
                ~train_clock.stop;
            })
        });
    """)    
    
def play_task_2h_sc():
    """Starts a two-hands task playthrough.
    
    Uses the TempoClock defined in previous function to schedule
    events for left and right hand.
    Starts the metronome and activates interactive sonification feedback.
    """
    
    sc.lang.cmds(r"""
        ~train_clock.schedAbs(~train_clock.beats.ceil, {
            ~train_inputs_left = ([]!~number_all_notes_left); ~train_inputs_right = ([]!~number_all_notes_right);
            ~solutions_time_left = ~timing_all_notes_left + ((~train_clock.beats + (^wait_bars * ^beats_per_bar)) ! ~number_all_notes_left);
            ~solutions_time_right = ~timing_all_notes_right + ((~train_clock.beats + (^wait_bars * ^beats_per_bar)) ! ~number_all_notes_right);
            ~solutions_note_left = Pseq(^train_notes_left, inf).asStream.nextN(~number_all_notes_left);
            ~solutions_note_right = Pseq(^train_notes_right, inf).asStream.nextN(~number_all_notes_right);
            ~metronome.play(~train_clock);
            if((^task_mode != "test"), {
                ~train_clock.schedAbs(~train_clock.beats + (^wait_bars * ^beats_per_bar) + ^train_offbeat_left + ^allow_error, {
                    ~control_task_left.play(~train_clock);
                });
                ~train_clock.schedAbs(~train_clock.beats + (^wait_bars * ^beats_per_bar) + ^train_offbeat_right + ^allow_error, {
                    ~control_task_right.play(~train_clock);
                })
            });
            ~train_clock.sched((^wait_bars + ^train_bars) * ^beats_per_bar - 0.01, {
                ~metronome.stop;
            }); 
            ~train_clock.sched((^wait_bars + ^train_bars) * ^beats_per_bar + ^allow_error - 0.01, {
                ~control_task_left.stop; ~control_task_right.stop;
                ~setup_default_note_on.value;
                ~train_clock.stop;
            })
        });
    """)

def collect_training_data(participant_id, training_data_list, task_name, task, method, bpm, run_number):
    """Retrieves playthrough information stored in global parameters
    in the SuperCollider environment. Creates a TrainingData object
    and saves the information inside it.
    
    Parameters
    ----------
    participant_id: str
        a unique random id for naming the save file
    training_data_list: list of TrainingData
        a list to which the created TrainingData will be added
    task_name: str
        name of the TrainingTask that is played in this playthrough
    task: TrainingTask
        the TrainingTask object named by task_name
    method: int
        index of sonification method used in this playthrough
    bpm: int
        tempo (in beats-per-minute) to be followed in this playthrough
    run_number: int
        index of the playthrough
    """
    
    if task.hands == 1:
        solutions_note_all = %scgs ~solutions_note
        solutions_time_all = %scgs ~solutions_time
        train_inputs_all = %scgs ~train_inputs
    elif task.hands == 2:
        solutions_note_left = %scgs ~solutions_note_left
        solutions_note_right = %scgs ~solutions_note_right
        solutions_time_left = %scgs ~solutions_time_left
        solutions_time_right = %scgs ~solutions_time_right
        train_inputs_left = %scgs ~train_inputs_left
        train_inputs_right = %scgs ~train_inputs_right
        solutions_note_all = [solutions_note_left, solutions_note_right]
        solutions_time_all = [solutions_time_left, solutions_time_right]
        train_inputs_all = [train_inputs_left, train_inputs_right]
    
    new_training_data = TrainingData(task_name, task, method, bpm, run_number, solutions_note_all, solutions_time_all, train_inputs_all)
    training_data_list.append(new_training_data)
    save_data_to_file(new_training_data, './' + participant_id, task_name)

def save_data_to_file(data, path, name):
    """Saves data to a file via pickle.
    
    Parameters
    ----------
    data: any
        data to be saved
    path: str
        path to the directory where the file is saved
    name: str
        name for the created file
    """
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    f = open(path + '/' + name + '_' + time.strftime('%b%d%Y_%H%M%S', time.localtime()), 'wb')
    pickle.dump(data, f)
    f.close()

def load_data_from_file(filename):
    """Loads data from a file via pickle.
    
    Parameters
    ----------
    filename: str
        name of the file to be read
    """
    
    result = None
    with (open(filename, "rb")) as f:
        result = pickle.load(f)
    return result

##### GUI elements

In [8]:
import ipywidgets as widgets
import time
from time import sleep

############################
### FUNCTION DEFINITIONS ###
############################
def demo_sound(b):
    """Plays a default piano note in middle c for 1.5 seconds"""
    
    %scs x = Synth(\train_note, [\freq, 60.midicps])
    sleep(1.5)
    %scs x.release

def test_sound_method1_early(b):
    """Plays a default piano note in middle c with the first
    sonification feedback method, treated as a note played
    too early for half second.
    """
    
    %scs x = Synth(\train_note, [\freq, 60.midicps, \distortion_length, 0.5, \amp_reduction_level, 0.8])
    sleep(1.5)
    %scs x.release

def test_sound_method1_late(b):
    """Plays a signal sound as described in the first
    sonification feedback method.
    """
    
    %scs Synth(\default_note, [\freq, 60.midicps])

def test_sound_method2_early(b):
    """Plays a default piano note in middle c with the second
    sonification feedback method, treated as a note played
    too early for half second.
    """
    
    %scs x = Synth(\train_note, [\freq, 60.midicps, \distortion_length, 0.5, \freq_distort_level, 1])
    sleep(1.5)
    %scs x.release

def test_sound_method2_late(b):
    """Plays a signal sound as described in the second
    sonification feedback method.
    """
    
    %scs Synth(\blip, [\freq, 60.midicps])
    
def test_sound_method3_early(b):
    """Plays a default piano note in middle c with the third
    sonification feedback method, treated as a note played
    too early for half second.
    """
    
    %scs x = Synth(\train_note, [\freq, 60.midicps, \distortion_length, 0.5, \noise_level, 1])
    sleep(1.5)
    %scs x.release

def test_sound_method3_late(b):  
    """Plays a signal sound as described in the third
    sonification feedback method.
    """
    
    %scs Synth(\tick)
    
#################
### GUI SETUP ###
#################    
play_demo_note = widgets.Button(description='Play demo note')
play_demo_note.on_click(demo_sound)

method1_early = widgets.Button(description='Early')
method1_early.on_click(test_sound_method1_early)
method1_late = widgets.Button(description='Late')
method1_late.on_click(test_sound_method1_late)

method2_early = widgets.Button(description='Early')
method2_early.on_click(test_sound_method2_early)
method2_late = widgets.Button(description='Late')
method2_late.on_click(test_sound_method2_late)

method3_early = widgets.Button(description='Early')
method3_early.on_click(test_sound_method3_early)
method3_late = widgets.Button(description='Late')
method3_late.on_click(test_sound_method3_late)

demo_sound_panel = widgets.GridBox([widgets.HTML('<b>Demo note:</b>'), play_demo_note], layout=widgets.Layout(grid_template_columns='repeat(2, 200px)'))
test_sounds_panel = widgets.GridBox([
    widgets.HTML('<b>Method 1:</b>'), method1_early, method1_late,
    widgets.HTML('<b>Method 2:</b>'), method2_early, method2_late,
    widgets.HTML('<b>Method 3:</b>'), method3_early, method3_late,], layout=widgets.Layout(grid_template_columns='repeat(3, 200px)'))
test_sounds_menu = widgets.VBox([demo_sound_panel, test_sounds_panel])

In [9]:
import ipywidgets as widgets
from fractions import Fraction
import time
import random
import string

participant_id = 'dummy'
task_list = {}
training_data = []
method_list = [('No method', 0), ('Method 1', 1), ('Method 2', 2), ('Method 3', 3)]
experiment_tasks_hard = {
                    'Task 0': TrainingTask('4/4', 1, [60], [1/2, 1, 1/2, 2], 0),
                    'Task 1': TrainingTask('4/4', 1, [60], [1, 1/2, 3/2, 1], 0),
                    'Task 2': TrainingTask('4/4', 1, [60], [1/2, 1, 1, 3/2], 0.5),
                    'Task 3': TrainingTask('4/4', 1, [60], [1, 1/2, 1/2, 2], 1)}

experiment_tasks_easy = {
                    'Task 0': TrainingTask('4/4', 1, [60], [1, 1, 2], 0),
                    'Task 1': TrainingTask('4/4', 1, [60], [1, 2, 1], 0),
                    'Task 2': TrainingTask('4/4', 1, [60], [2, 1, 1], 0),
                    'Task 3': TrainingTask('4/4', 1, [60], [1, 1, 2], 1)}

experiment_bpm = 60
experiment_wait_bars = 1
experiment_train_bars = 4
experiment_allow_error = 0.05
runs_per_routine = 2
current_run_number = 0
experiment_mode = False

############################
### FUNCTION DEFINITIONS ###
############################

# General functions used by the GUI
def get_playthrough_duration():
    """Returns playthrough duration.
    
    Returns
    -------
    float
        length of the full playthrough in seconds
    """
    return 60/bpm_setting.value * (wait_bars_setting.value+train_bars_setting.value) * int(signature_setting.value.split('/')[0])

def play_task(mode):
    """Retrieves settings, sets up and starts a training task.
    
    Parameters
    ----------
    mode: str
        the training mode in this playthrough,
        it can be 'demo', 'test' or 'train'
            demo: only listen to the rhythm
            test: perform the rhythm without help
            train: train the rhythm with (or without) auditory feedback
    """
    
    settings = {'task_mode': mode,
                'bpm': bpm_setting.value,
                'wait_bars': wait_bars_setting.value,
                'train_bars': train_bars_setting.value,
                'allow_error': allow_error_setting.value
               }
    if mode == 'demo':
        settings.update({'allow_error': 0})
    setup_task_sc(task_list[select_task.value], settings, select_method.value)
    play_task_sc(task_list[select_task.value])

def disable_settings():
    """Disables task settings and general settings in the GUI."""
    
    add_save_task_button.disabled = True
    delete_task_button.disabled = True
    select_task.disabled = True
    select_method.disabled = True
    bpm_setting.disabled = True
    wait_bars_setting.disabled = True
    train_bars_setting.disabled = True
    allow_error_setting.disabled = True
    
def enable_settings():
    """Enables task settings and general settings in the GUI."""
    
    add_save_task_button.disabled = False
    delete_task_button.disabled = False
    select_task.disabled = False
    select_method.disabled = False
    bpm_setting.disabled = False
    wait_bars_setting.disabled = False
    train_bars_setting.disabled = False
    allow_error_setting.disabled = False
    
def disable_plays():
    """Disable buttons in the GUI"""
    
    play_demo_button.disabled = True
    start_test_button.disabled = True
    start_training_button.disabled = True
        
def enable_plays():
    """Enable buttons in the GUI"""
    
    play_demo_button.disabled = False
    start_test_button.disabled = False
    start_training_button.disabled = False
    
    
# Functions for controlling the experiment procedure
def start_new_experiment(experiment_tasks):
    """Starts a new experiment with a set of experiment tasks.
    
    Sets up and re-initializes global parameters.
    
    Parameters
    ----------
    experiment_tasks: list of TrainingTask
        list of training tasks to be performed in this experiment
    """
    
    global task_list, training_data, experiment_mode
    task_list = experiment_tasks
    training_data = []
    disable_settings()
    experiment_mode = True
    
def start_new_routine():
    """Starts a new training routine for the current task selected.
    
    Re-initializes global parameters and adjusts GUI elements.
    """
    
    global current_run_number
    disable_plays()
    current_run_number = 0
    run_counter.value = '1. test'
    play_demo_button.disabled = False
    start_test_button.disabled = False
    next_task_button.disabled = True

def proceed_to_next_phase(last_action):
    """Updates the state after a playthrough is completed.
    
    Keeps track of the total number of playthroughs performed
    for the current task selected.
    Switches to training phase if the 1. testing phase is
    completed, switches to 2. testing phase if the training
    phase is completed, etc.
    
    Parameters
    ----------
    last_action: str
        type of the last playthrough that just completed
    """
    
    global current_run_number
    if last_action == 'Play Demo':
        play_demo_button.disabled = False
        if current_run_number == 0 or current_run_number == runs_per_routine + 1:
            start_test_button.disabled = False
        else:
            start_training_button.disabled = False
    elif last_action == 'Start Test':
        if current_run_number == 0:
            play_demo_button.disabled = False
            start_training_button.disabled = False
            current_run_number += 1
            run_counter.value = str(current_run_number)
        elif select_task.index < len(select_task.options) - 1:     # End of routine reached
            next_task_button.disabled = False
            run_counter.value = 'End of this routine. Please proceed to the next task.'
        else:                                                      # End of whole experiment reached
            run_counter.value = 'End of this experiment. You can see results or quit.'
            view_results_button.disabled = False
    elif last_action == 'Start Training':
        play_demo_button.disabled = False
        if current_run_number < runs_per_routine:
            start_training_button.disabled = False
            current_run_number += 1
            run_counter.value = str(current_run_number)
        else:
            start_test_button.disabled = False
            current_run_number += 1
            run_counter.value = '2. test'
    
def quit_experiment():
    """Quits the current experiment."""
    
    global task_list, training_data, experiment_mode
    task_list = {}
    training_data = []
    enable_settings()
    enable_plays()
    experiment_mode = False   
    
    
# Event functions
def add_save_task_from_gui(b):
    """Adds or saves a task with parameters defined in task settings
    to the global task list.
    """
    
    if hand_settings.selected_index == 0:
        task_list.update({task_name.value : TrainingTask(
            signature_setting.value, 1, 
            [int(x) for x in notes_setting.value.split(',')], 
            [float(Fraction(x)) for x in rhythms_setting.value.split(',')],
            float(Fraction(offbeat_setting.value))
        )})
    else:
        task_list.update({task_name.value : TrainingTask(
            signature_setting.value, 2,
            [[int(x) for x in note_left_setting.value.split(',')], [int(x) for x in note_right_setting.value.split(',')]],
            [[float(Fraction(x)) for x in rhythm_left_setting.value.split(',')], [float(Fraction(x)) for x in rhythm_right_setting.value.split(',')]],
            [float(Fraction(offbeat_left_setting.value)), float(Fraction(offbeat_right_setting.value))]
        )})
    select_task.options = task_list.keys()

def delete_task_from_gui(b):
    """Deletes the task with task name given in the task settings
    from the global task list.
    """
    
    task_list.pop(task_name.value)
    select_task.options = task_list.keys()

def play_task_from_gui(b): 
    """Starts a playthrough with the currently selected task.
    
    Depending on which button is pressed, the playthrough will
    be performed with one of the following modes:
        demo: only listen to the rhythm
        test: perform the rhythm without help
        train: train the rhythm with (or without) auditory feedback
    """
    
    if select_task.value != None: 
        if b.description == 'Play Demo':
            play_task('demo')
        elif b.description == 'Start Test':
            play_task('test')
        elif b.description == 'Start Training':
            if select_method.value == None: return
            else: play_task('train')
        playthrough_duration = get_playthrough_duration() 
        disable_plays()
        time.sleep(playthrough_duration + 1)
        if b.description != 'Play Demo':
            collect_training_data(participant_id, training_data, select_task.value, task_list[select_task.value], 
                                  select_method.value, bpm_setting.value, current_run_number)
        if experiment_mode == False:
            enable_plays()
        else:
            proceed_to_next_phase(b.description)

def start_experiment_from_gui(b):
    """Creates a new participant id and starts a new experiment."""
    
    global runs_per_routine, participant_id
    participant_id = ''.join([random.choice(string.ascii_letters + string.digits) for n in range(8)])
    if hard_mode_toggle.value == True:
        start_new_experiment(experiment_tasks_hard)
        runs_per_routine = 3
    else:     
        start_new_experiment(experiment_tasks_easy)
        runs_per_routine = 4
    bpm_setting.value = experiment_bpm
    wait_bars_setting.value = experiment_wait_bars
    train_bars_setting.value = experiment_train_bars
    allow_error_setting.value = experiment_allow_error
    select_task.options = task_list.keys()
    select_task.index = 0
    select_method.index = 0
    start_new_routine()
    start_experiment_button.disabled = True
    quit_experiment_button.disabled = False

def proceed_to_next_task(b):
    """Selects the next task (if there are any) in the task list
    and starts a new training routine. This method is only used
    during an experiment.
    """
    if select_task.index < len(select_task.options):
        select_task.index += 1
        select_method.index += 1
        start_new_routine()

def quit_experiment_from_gui(b):
    """Quit the current experiment."""
    
    quit_experiment()
    start_experiment_button.disabled = False
    quit_experiment_button.disabled = True
    view_results_button.disabled = True
    select_task.options = task_list.keys()

def view_results(b):
    """Saves training data to file and plots the result."""
    
    if len(training_data) > 0:
        save_data_to_file(training_data, './experiment_data', participant_id)
        plot_all_training_data(training_data)

#################
### GUI SETUP ###
#################

# Task Settings
signature_setting = widgets.Combobox(options=[('2/4'), ('3/4'), ('4/4')],value='4/4')


# Settings for 1-hand tasks
notes_setting = widgets.Text(value='60')
rhythms_setting = widgets.Text(value='1')
offbeat_setting = widgets.Text(value='0', layout=widgets.Layout(width='50px'))

one_hand_settings = widgets.GridBox([
    widgets.Label('Notes:'), notes_setting,
    widgets.Label('Rhythm:'), rhythms_setting,
    widgets.Label('Offbeat:'), offbeat_setting
    ], layout=widgets.Layout(grid_template_columns='repeat(6, auto)'))


# Settings for 2-hands tasks
note_left_setting = widgets.Text(value='48')
note_right_setting = widgets.Text(value='60')

rhythm_left_setting = widgets.Text(value='1')
rhythm_right_setting = widgets.Text(value='1')

offbeat_left_setting = widgets.Text(value='0', layout=widgets.Layout(width='50px'))
offbeat_right_setting = widgets.Text(value='0', layout=widgets.Layout(width='50px'))

info_two_hands = widgets.HTML(value='<p><b>Training with two hands only supports tapping a single note per hand.</b><br>Please assign two different notes for the right and the left hand. (Default settings are C for the right hand and CC for the left hand.)</p>')

two_hands_settings = widgets.GridBox([
    widgets.Label('Left hand note:'), note_left_setting,
    widgets.Label('Right hand note:'), note_right_setting,
    widgets.Label('Left hand rhythm:'), rhythm_left_setting,
    widgets.Label('Right hand rhythm:'), rhythm_right_setting,
    widgets.Label('Left hand offbeat:'), offbeat_left_setting,
    widgets.Label('Right hand offbeat:'), offbeat_right_setting,
    ], layout=widgets.Layout(grid_template_columns='repeat(4, auto)'))


# Combining settings into one tab
hand_settings = widgets.Tab(children=[one_hand_settings, widgets.VBox([info_two_hands, two_hands_settings])], selected_index=0)
hand_settings.set_title(0, '1-hand')
hand_settings.set_title(1, '2-hands')


# Add, save and delete task
task_name = widgets.Text()

add_save_task_button = widgets.Button(
    description='Add / Save Task',
    button_style='info',
    tooltip='Create a new task or update the task with the given name',
    icon='plus'
)
add_save_task_button.on_click(add_save_task_from_gui)

delete_task_button = widgets.Button(
    description='Delete Task',
    button_style='danger',
    tooltip='Delete the task with the given name',
    icon='times'
)
delete_task_button.on_click(delete_task_from_gui)

task_settings = widgets.VBox([widgets.HBox([widgets.Label('Signature:'), signature_setting,]),
                              hand_settings, 
                              widgets.HBox([widgets.Label('Name:'), task_name, add_save_task_button, delete_task_button])
                             ])


# General Settings
bpm_setting = widgets.IntSlider(value=60, min=30, max=200, description='BPM:')
wait_bars_setting = widgets.IntSlider(value=1, min=1, max=4, description='Wait:')
train_bars_setting = widgets.IntSlider(value=1, min=1, max=10, description='Train:')
allow_error_setting = widgets.FloatSlider(value=0.05, step=0.01, min=0, max=0.5, description='Allow error:')

general_settings = widgets.VBox([bpm_setting, 
                                 widgets.HBox([wait_bars_setting, widgets.Label('bars')]), 
                                 widgets.HBox([train_bars_setting, widgets.Label('bars')]),
                                 widgets.HBox([allow_error_setting, widgets.Label('beat')])
                                ])


# Combining settings into one accordion
settings = widgets.Accordion(children=[task_settings, general_settings], selected_index=None)
settings.set_title(0, 'Task Settings')
settings.set_title(1, 'General Settings')


# Main layout for testing and training
play_demo_button = widgets.Button(
    description='Play Demo',
    button_style='',
    tooltip='Hear the full task',
    icon='music'
)
play_demo_button.on_click(play_task_from_gui)

start_test_button = widgets.Button(
    description='Start Test',
    button_style='info',
    tooltip='Start the test',
    icon='play'
)
start_test_button.on_click(play_task_from_gui)

start_training_button = widgets.Button(
    description='Start Training',
    button_style='warning',
    tooltip='Start training session for the task',
    icon='dumbbell'
)
start_training_button.on_click(play_task_from_gui)

next_task_button = widgets.Button(
    description='Next Task',
    button_style='success',
    tooltip='Proceed to the next task',
    icon='forward',
    disabled=True
)
next_task_button.on_click(proceed_to_next_task)

hard_mode_toggle = widgets.ToggleButton(
    value=False,
    description='Hard Mode',
    disabled=False,
    button_style='',
    tooltip='Toggle to start an experiment with more complex tasks',
    icon='bomb'
)

start_experiment_button = widgets.Button(
    description='Start New Experiment',
    button_style='success',
    tooltip='Play through a serie of experiment tasks',
    icon='plus',
    layout=widgets.Layout(width='auto')
)
start_experiment_button.on_click(start_experiment_from_gui)

quit_experiment_button = widgets.Button(
    description='Quit Experiment',
    button_style='danger',
    tooltip='Quit experiment',
    icon='door-open',
    disabled=True
)
quit_experiment_button.on_click(quit_experiment_from_gui)

view_results_button = widgets.Button(
    description='View Results',
    button_style='info',
    tooltip='Quit experiment',
    icon='chart-line',
    disabled=True
)
view_results_button.on_click(view_results)

select_task = widgets.Dropdown(options=task_list.keys(), description="Task:")
select_method = widgets.Dropdown(options=method_list, description="Method:")
run_counter = widgets.Label(str(current_run_number))

api_gui = widgets.VBox([settings, 
                        widgets.HBox([hard_mode_toggle, start_experiment_button, quit_experiment_button, view_results_button]), 
                        widgets.HBox([select_task, play_demo_button, start_test_button, next_task_button]), 
                        widgets.HBox([select_method, start_training_button, widgets.Label('Current run:'), run_counter])])

In [10]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

sorted_training_data = {}
round_decimal = 6
input_selected = []
input_selected_fig = None
input_selected_ax = None
ax_err_ylim = [-0.5, 1]

############################
### FUNCTION DEFINITIONS ###
############################

# General functions used by the GUI
def compute_absolute_training_errors(solutions, train_inputs, missed_input_error):
    """Computes absolute time errors between a list of solutions
    and a list of input values.
    
    Parameters
    ----------
    solutions: list of float
        a list of time values indicating the correct time for each note
        in a playthrough
    train_inputs: list of list
        a list of all inputs made by the player
    missed_input_error: float
        a default error value to be added when the player has missed
        the note completely
    
    Returns
    -------
    absolutes: list of float
        a list of absolute time errors
    """
    
    absolutes = [[] for _ in range(len(solutions))]
    for train_input in train_inputs:
        index = int(train_input[2])
        absolute_error = np.abs(train_input[1] - solutions[index])
        absolutes[index].append(absolute_error)
    for result in absolutes:
        if len(result) == 0:     # There is no input for this note
            result.append(missed_input_error)
        elif len(result) > 1:    # There are multiple inputs registered for this note
            result_mean = np.mean(result)
            result.clear()
            result.append(result_mean)
    absolutes = np.array(absolutes).flatten()
    return absolutes
    
def compute_relative_training_errors(solutions, train_inputs, missed_input_error):
    solutions_relatives = np.diff(solutions)
    relatives = [[] for _ in range(len(solutions_relatives))]
    time_last_notes = [[] for _ in range(len(solutions))]
    for train_input in train_inputs:
        index = int(train_input[2])
        time_last_notes[index].append(train_input[1])
        if index > 0:
            for time in time_last_notes[index-1]:
                relative_distance = train_input[1] - time
                relative_error = np.abs(relative_distance - solutions_relatives[index-1])
                relatives[index-1].append(relative_error)
    for result in relatives:
        if len(result) == 0:     # There is no input for this note
            result.append(missed_input_error)
        elif len(result) > 1:    # There are multiple inputs registered for this note
            result_mean = np.mean(result)
            result.clear()
            result.append(result_mean)   
    relatives = np.array(relatives).flatten()
    return relatives
    
def compute_errors(training_data):
    """Computes mean and standard deviation of absloute and relative 
    time errors for a set of training data, then saves the results
    in the TrainingData object as attributes.
    
    Parameters
    ----------
    training_data: TrainingData
        a set of data to be analyzed
    """
    absolute_errors = compute_absolute_training_errors(training_data.solutions_time, training_data.train_inputs, 0.5)
    relative_errors = compute_relative_training_errors(training_data.solutions_time, training_data.train_inputs, 0.5)
    mean_abs = np.around(np.mean(absolute_errors), round_decimal)
    std_abs = np.around(np.std(absolute_errors), round_decimal)
    mean_rel = np.around(np.mean(relative_errors), round_decimal)
    std_rel = np.around(np.std(relative_errors), round_decimal)
    training_data.absolute_errors = absolute_errors
    training_data.relative_errors = relative_errors
    training_data.mean_abs = mean_abs
    training_data.std_abs = std_abs
    training_data.mean_rel = mean_rel
    training_data.std_rel = std_rel
    
def sort_all_training_data(training_data_list):
    """Sorts a list of TrainingData into a dictionary
    with task names as keys and a list of all training data
    belonging to the task as their value.
    
    Parameters
    ----------
    training_data_list: list of TrainingData
        a list of training data to be sorted and sorted
    
    Retunrs
    -------
    result: dict
        a dictionary with sorted training data
    """
    
    result = {}
    for data in training_data_list:
        compute_errors(data)
        if not data.task_name in result:
            task_name = data.task_name
            result.update({task_name: []})
        result[task_name].append(data)
    for task_name in result:
        result[task_name].sort(key=lambda training_data: training_data.run_number)
    return result
            
def plot_all_training_data(training_data_list):
    """Creates plots given a list of training data. Groups the data
    by their tasks and plots training data for the first task.
    
    Parameters
    ----------
    training_data_list: list of TrainingData
        a list of training data to be sorted and plotted
    """
    
    assert len(training_data_list) > 0, 'No training data provided.'
    global sorted_training_data
    sorted_training_data = sort_all_training_data(training_data_list)
    select_task_data.options = sorted_training_data.keys()
    select_task_data.index = 0
    plot_full_task(select_task_data.value) 

def plot_full_task(task_name):
    """Plots training data for first testing phase, training phase
    and second testing phase onto respective figures and axes for a
    given task.
    
    Parameters
    ----------
    task_name: str
        the name of the training task to be plotted.
    """
    
    training_data_list = sorted_training_data[task_name]
    global fig_test1, fig_train, fig_test2, ax_test1, ax_train, ax_test2
    plot_training_data(fig_test1, ax_test1, training_data_list[0])
    if len(training_data_list) > 2:
        plot_training_data(fig_train, ax_train, training_data_list[1])
        select_run.value = 1
        select_run.max = len(training_data_list) - 2
        plot_training_data(fig_test2, ax_test2, training_data_list[-1])
    plot_errors_for_task(task_name)

def plot_training_data(fig, ax, training_data):
    """Plots training data onto selected axes and redraws the
    corresponding figure to display the changes.
    
    The correct notes recreated from note solutions and time
    solutions are plotted as crosses, the notes played by
    the player are plotted as star shaped points.
    
    Parameters
    ----------
    fig: matplotlib.pyplot.figure
        the figure that will be updated
    ax: matplotlib.axes
        the axes that will be updated
    training_data: TrainingData
        the training data to be plotted on ax
    """
    
    clear_axes(ax)
    ax.grid(axis='x')
    start_beat = np.floor(training_data.solutions_time[0])
    x_solutions = training_data.solutions_time - start_beat
    y_solutions = training_data.solutions_note
    inputs = np.hsplit(training_data.train_inputs, 3)
    x_inputs = inputs[1].flatten() - start_beat
    y_inputs = inputs[0].flatten()
    ax.plot(x_solutions, y_solutions, 'x')
    ax.plot(x_inputs, y_inputs, '*', picker=5)
    fig.canvas.draw()
    update_errors_display(ax, training_data.mean_abs, training_data.std_abs, training_data.mean_rel, training_data.std_rel)
    
def update_errors_display(ax, mean_abs, std_abs, mean_rel, std_rel):
    """Updates labels to display new error values.
    
    Parameters
    ----------
    ax: matplotlib.axes
        the axes with updated data
    mean_abs: float
        new mean of absolute time errorr
    std_abs: float
        new standard deviation of absolute time errors
    mean_rel: float
        new mean of relative time errorr
    std_rel: float
        new standard deviation of relative time errors
    """
    
    if ax == ax_test1:
        absolute_error_test1.value = 'mean: ' + str(mean_abs) + '; std: ' + str(std_abs)
        relative_error_test1.value = 'mean: ' + str(mean_rel) + '; std: ' + str(std_rel)
    elif ax == ax_train:
        absolute_error_train.value = 'mean: ' + str(mean_abs) + '; std: ' + str(std_abs)
        relative_error_train.value = 'mean: ' + str(mean_rel) + '; std: ' + str(std_rel)
    elif ax == ax_test2:
        absolute_error_test2.value = 'mean: ' + str(mean_abs) + '; std: ' + str(std_abs)
        relative_error_test2.value = 'mean: ' + str(mean_rel) + '; std: ' + str(std_rel)

def plot_errors_for_task(task_name):
    """Plots the average absolute and relative time errors
    for each playthrough in a task as errorbar diagrams.
    
    Parameters
    ----------
    task_name: str
        the name of the training task to be plotted.
    """
    
    global fig_abserr, ax_abserr, fig_relerr, ax_relerr
    training_data_list = sorted_training_data[task_name]
    attempts = range(len(training_data_list))
    means_abs = [data.mean_abs for data in training_data_list]
    stds_abs = [data.std_abs for data in training_data_list]
    means_rel = [data.mean_rel for data in training_data_list]
    stds_rel = [data.std_rel for data in training_data_list]
    clear_axes(ax_abserr)
    ax_abserr.axhline(color='black')
    ax_abserr.set_ylim(ax_err_ylim)
    ax_abserr.set_xticks(attempts)
    ax_abserr.errorbar(attempts, means_abs, stds_abs, fmt='o', capsize=6)
    fig_abserr.canvas.draw()
    clear_axes(ax_relerr)
    ax_relerr.axhline(color='black')
    ax_relerr.set_ylim(ax_err_ylim)
    ax_relerr.set_xticks(attempts)
    ax_relerr.errorbar(attempts, means_rel, stds_rel, fmt='o', capsize=6)
    fig_relerr.canvas.draw()
    
def clear_axes(ax):
    """Clears the content of selected axes.
    
    Parameters
    ----------
    ax: matplotlib.axes
        the axes to be cleared
    """
    
    ax_title = ax.get_title()
    ax_xlabel = ax.get_xlabel()
    ax_ylabel = ax.get_ylabel()
    ax.clear()
    ax.set_title(ax_title)
    ax.set_xlabel(ax_xlabel)
    ax.set_ylabel(ax_ylabel)
    
def clear_all():
    """Clears and redraws all axes."""
    
    clear_axes(ax_test1)
    ax_test1.grid(axis='x')
    clear_axes(ax_train)
    ax_train.grid(axis='x')
    clear_axes(ax_test2)
    ax_test2.grid(axis='x')
    clear_axes(ax_abserr)
    ax_abserr.axhline(color='black')
    ax_abserr.set_ylim([-0.5, 1])
    clear_axes(ax_relerr)
    ax_relerr.axhline(color='black')
    ax_relerr.set_ylim([-0.5, 1])
    fig_test1.canvas.draw()
    fig_train.canvas.draw()
    fig_test2.canvas.draw()
    fig_abserr.canvas.draw()
    fig_relerr.canvas.draw()

# Event functions
def update_training_run_plot(change):
    """Re-plots the training run figure with new training data
    when a training run with another index is selected.
    """
    
    training_data_list = sorted_training_data[select_task_data.value]
    plot_training_data(fig_train, ax_train, training_data_list[change.new])

def update_task_plot(change):
    """Re-plots all figures with training data from the new task
    when another training task is selected."""
    
    plot_full_task(change.new)

def note_index_changed(change):
    """Enables the change index button"""
    
    update_note_index_button.disabled = False

def update_note_index_for_input_selected(b):
    """Changes the note index of a selecetd input note
    and re-plots absolute and relative errors.
    """
    
    new_index = int(select_note_index.value)
    training_data = input_selected[0]
    input_index = input_selected[1]
    training_data.train_inputs[input_index][2] = new_index
    compute_errors(training_data)
    plot_training_data(input_selected_fig, input_selected_ax, training_data)
    plot_errors_for_task(select_task_data.value)
    update_note_index_button.disabled = True

def on_pick(event):
    """An on pick event for when the player selects a
    data point, that indicates a note played by them,
    with the mouse.
    
    The selected note is registered in a global variable
    so that it can be modified when the player wishies to
    change its properties.
    """
    
    global input_selected, input_selected_fig, input_selected_ax
    if isinstance(event.artist, Line2D):
        thisline = event.artist
        xdata = thisline.get_xdata()
        ydata = thisline.get_ydata()
        ind = event.ind
        if thisline in ax_test1.lines:
            training_data_index = 0
            input_selected_fig = fig_test1
            input_selected_ax = ax_test1
        elif thisline in ax_train.lines:
            training_data_index = select_run.value
            input_selected_fig = fig_train
            input_selected_ax = ax_train
        elif thisline in ax_test2.lines:
            training_data_index = -1
            input_selected_fig = fig_test2
            input_selected_ax = ax_test2
        training_data = sorted_training_data[select_task_data.value][training_data_index]
        input_selected = [training_data, ind[0]]
        select_note_index.options = list(range(len(training_data.solutions_time)))
        select_note_index.index = int(training_data.train_inputs[ind[0]][2])
        input_selected_description.value = '[' + str(np.around(np.take(xdata, ind)[0], 2)) + ', ' + str(int(np.take(ydata, ind))) + ']'
        update_note_index_button.disabled = True
        

    
#################
### GUI SETUP ###
#################

# Setup GUI for plotting results   
plt.ioff()

select_task_data = widgets.Dropdown(options=sorted_training_data.keys(), description="Task:")
select_task_data.observe(update_task_plot, names='value')

input_selected_description = widgets.Label('none')
select_note_index = widgets.Dropdown(options=[], layout=widgets.Layout(width='50px'))
select_note_index.observe(note_index_changed, names='value')
update_note_index_button = widgets.Button(
    description='Update Index',
    button_style='',
    tooltip='Change the note to which this input belongs',
    icon='',
    disabled=True
)
update_note_index_button.on_click(update_note_index_for_input_selected)

select_run = widgets.IntSlider(
    description='Train run:',
    value=1,
    min=1,
    max=3
)
select_run.observe(update_training_run_plot, names='value')

absolute_error_test1 = widgets.Label(value='mean: N/A; std: N/A')
relative_error_test1 = widgets.Label(value='mean: N/A; std: N/A')
absolute_error_train = widgets.Label(value='mean: N/A; std: N/A')
relative_error_train = widgets.Label(value='mean: N/A; std: N/A')
absolute_error_test2 = widgets.Label(value='mean: N/A; std: N/A')
relative_error_test2 = widgets.Label(value='mean: N/A; std: N/A')

errors_test1 = widgets.VBox([widgets.HBox([widgets.Label('Absolute error'), absolute_error_test1]),
                             widgets.HBox([widgets.Label('Relative error'), relative_error_test1])])
errors_train = widgets.VBox([widgets.HBox([widgets.Label('Absolute error'), absolute_error_train]),
                             widgets.HBox([widgets.Label('Relative error'), relative_error_train])])
errors_test2 = widgets.VBox([widgets.HBox([widgets.Label('Absolute error'), absolute_error_test2]),
                             widgets.HBox([widgets.Label('Relative error'), relative_error_test2])])


# Plot figure for the 1. testing phase
fig_test1, ax_test1 = plt.subplots()
fig_test1.canvas.header_visible = False
ax_test1.grid(axis='x')
ax_test1.set_title('1. Test Run')
ax_test1.set_xlabel('Time in Beats')
ax_test1.set_ylabel('Note in MIDI')
fig_test1.canvas.mpl_connect('pick_event', on_pick)

# Plot figure for the training phase
fig_train, ax_train = plt.subplots()
fig_train.canvas.header_visible = False
ax_train.grid(axis='x')
ax_train.set_title('Training Runs')
ax_train.set_xlabel('Time in Beats')
ax_train.set_ylabel('Note in MIDI')
fig_train.canvas.mpl_connect('pick_event', on_pick)

# Plot figure for the 2. testing phase
fig_test2, ax_test2 = plt.subplots()
fig_test2.canvas.header_visible = False
ax_test2.grid(axis='x')
ax_test2.set_title('2. Test Run')
ax_test2.set_xlabel('Time in Beats')
ax_test2.set_ylabel('Note in MIDI')
fig_test2.canvas.mpl_connect('pick_event', on_pick)

#Plot figure for absloute time errors
fig_abserr, ax_abserr = plt.subplots()
fig_abserr.canvas.header_visible = False
ax_abserr.axhline(color='black')
ax_abserr.set_ylim([-0.5, 1])
ax_abserr.set_title('Absolute Time Errors')
ax_abserr.set_xlabel('Attempts')
ax_abserr.set_ylabel('Error in Beats')

#Plot figure for relative time errors
fig_relerr, ax_relerr = plt.subplots()
fig_relerr.canvas.header_visible = False
ax_relerr.axhline(color='black')
ax_relerr.set_ylim([-0.5, 1])
ax_relerr.set_title('Relative Time Errors')
ax_relerr.set_xlabel('Attempts')
ax_relerr.set_ylabel('Error in Beats')


select_task_point = widgets.HBox([widgets.Label('Selected input:'), input_selected_description,
                                  widgets.Label('at note index'), select_note_index, update_note_index_button])

# Plotting results in 3 figures
results_plot = widgets.GridspecLayout(4, 2)
results_plot[0, 0] = fig_test1.canvas
results_plot[0, 1] = widgets.VBox([select_task_point, errors_test1], layout=widgets.Layout(height='auto', width='auto', justify_content='center'))
results_plot[1, 0] = fig_train.canvas
results_plot[1, 1] = widgets.VBox([select_run, select_task_point, errors_train], layout=widgets.Layout(height='auto', width='auto', justify_content='center'))
results_plot[2, 0] = fig_test2.canvas
results_plot[2, 1] = widgets.VBox([select_task_point, errors_test2], layout=widgets.Layout(height='auto', width='auto', justify_content='center'))
results_plot[3, 0] = fig_abserr.canvas
results_plot[3, 1] = fig_relerr.canvas

plot_gui = widgets.VBox([select_task_data, results_plot])
plot_gui.layout.align_items = 'center'

# rhythm_training
This application is dedicated to the accurate performing of a given rhythm. You can setup tasks to play after and hear real-time audio feedback while playing.

### Initialization
It is recommended to use JupyterLab to open this notebook. The following Python packages are required:
* [sc3nb](https://pypi.org/project/sc3nb/)
* [ipywidgets](https://pypi.org/project/ipywidgets/) (version 7.6.5 and later)
* [ipympl](https://matplotlib.org/ipympl/) for interactive matplotlib functions

Please make sure a MIDI keyboard is connected to the device this notebook is running on, then restart the kernel and run all cells.

## Free Play
You can set up and play training tasks manually.

### Task Settings
<b>Signature:</b> The desired time signature of the task. You can choose between 2/4, 3/4 and 4/4. Manual inputs should follow the same format.

<b>Notes:</b> A sequence of notes to be repeatedly played during the task. The values should be MIDI values: Middle C is 60, C# is 61, D is 62 etc. Notes are seperated by a single comma. E.g. a sequence C D E will be written as 60,62,64

Please note that this application only deals with rhythm and timing issues, so even if you can specify the notes you play, only temporal errors are processed.

<b>Rhythm:</b> A sequence of time intervals between the notes played during the task. The values should be relative to quarternote: a quarter note is 1, an eighth note is 1/2 etc. Intervals are seperated by a single comma. E.g., a sequence of one quarter note and two eighth notes will be written as 1,1/2,1/2

<b>Offbeat:</b> An offset at the beginning of the Task. Should be written as whole numbers or fractions (1/2 for half beat off).

<b>Name:</b> The name of the task. If the name already exist in the task list, the corresponding task in the task list will be updated.

### General Settings
<b>BPM:</b> Tempo of the task.

<b>Wait:</b> How many bars to wait before the task starts. The metronome will play during the waiting bars.

<b>Train:</b> How many bars to be played during the task.

<b>Allow error:</b> Input latency within this range will not be considered missing the note, so no control sound will be played to remind you to play the note.

### Start Test
Play the selected Task without any help. You can use Play Demo to hear the full task.

### Start Training
Play the selected Task with the selected sonification method. Different method will use different sound cues when you play the note at a wrong time.

## Methods
Three sets of audio feedback are available, each with a different style.

**Method 1:** When you play a note too early, the note will be dampened so that it sounds quieter. When you miss a note, the correct note will be played.

**Method 2:** When you play a note too early, the note will be distorted in frequency. When you miss a note, a blip sound will be played.

**Method 3:** When you play a note too early, you will hear a static noise. When you miss a note, a tick sound will be played.

You can listen to these sounds using the following panel. Middle C is the pitch for the demo note.

In [11]:
display(test_sounds_menu)

## Experiment
Play through a set of designed tasks. After you complete the experiment, you can view your training results in the following cell.

In [12]:
display(api_gui)

In [13]:
display(plot_gui)

## Exit
Run the following cell to exit the SC server.

In [14]:
def exit_program(b):
    plt.close(fig_test1)
    plt.close(fig_train)
    plt.close(fig_test2)
    plt.close(fig_abserr)
    plt.close(fig_relerr)
    sc.exit()

exit_program_button = widgets.Button(
    description='Exit',
    button_style='danger'
)
exit_program_button.on_click(exit_program)

exit_program_button

Button(button_style='danger', description='Exit', style=ButtonStyle())